https://medium.com/free-code-camp/big-picture-machine-learning-classifying-text-with-neural-networks-and-tensorflow-d94036ac2274

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from collections import Counter
from sklearn.datasets import fetch_20newsgroups

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

In [ ]:
my_graph = tf.Graph()
with tf.compat.v1.Session()  as sess:
    x = tf.constant([1,3,6])
    y = tf.constant([1,1,1])
    op = tf.add(x,y)
    result = sess.run(fetches=op)
    print(result)

In [ ]:
vocab = Counter()
text = "Hi from someone"
for word in text.split(' '):
    word_lowercase = word.lower()
    vocab[word_lowercase] += 1

def get_word_2_index(vocab):
    word2index = {}
    for i, word in enumerate(vocab):
        word2index[word] = i
    
    return word2index

In [ ]:
word2index = get_word_2_index(vocab)
total_words = len(vocab)
matrix = np.zeros((total_words), dtype = float)

for word in text.split():
    matrix[word2index[word.lower()]] += 1

print("Hi from someone:", matrix)

In [ ]:
matrix = np.zeros((total_words), dtype = float)
text = "Hi"
for word in text.split():
    matrix[word2index[word.lower()]] += 1
print("Hi:", matrix)

In [ ]:
categories = ['comp.graphics', 'sci.space','rec.sport.baseball']
newsgroups_train = fetch_20newsgroups(subset = 'train', categories = categories)
newsgroups_test = fetch_20newsgroups(subset = 'test', categories = categories)

In [ ]:
print('Total text in train:', len(newsgroups_train.data))
print('Total text in test:', len(newsgroups_test.data))

In [ ]:
print('text', newsgroups_train.data[0])
print('category:', newsgroups_train.target[0])

In [ ]:
vocab = Counter()

for text in newsgroups_train.data:
    for word in text.split(' '):
        vocab[word.lower()] += 1

for text in newsgroups_test.data:
    for word in text.split(' '):
        vocab[word.lower()] += 1

In [ ]:
print('Total words:', len(vocab))

In [ ]:
total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i, word in enumerate(vocab):
        word2index[word.lower()] = i
    return word2index
word2index = get_word_2_index(vocab)

print("Index of the word 'the':", word2index['the'])

In [ ]:
def text_to_vector(text):
    layer = np.zeros(total_words, dtype = float)
    for word in text.split(' '):
        layer[word2index[word.lower()]] += 1
    return layer

def category_to_vector(category):
    y = np.zeros((3), dtype = float)
    if category == 0:
        y[0] = 1.
    elif category == 1:
        y[1] == 1.
    else:
        y[2] = 1.
    return y

In [ ]:
def get_batch(df, i, batch_size):
    batches = []
    results = []
    texts = df.data[i * batch_size: i * batch_size + batch_size]
    categories = df.target[i * batch_size: i * batch_size + batch_size]
    
    for text in texts:
        layer = text_to_vector(text)
        batches.append(layer)
    
    for category in categories:
        y = category_to_vector(category)
        results.append(y)
    return np.array(batches), np.array(results)

In [ ]:
print("Each batch has 100 texts and each matrix has 119930 "
      "elements (word):", 
      get_batch(newsgroups_train, 1, 100)[0].shape)

In [ ]:
print("Each batch has 100 labels and each matrix has 3"
      "elements (3 categories):", 
      get_batch(newsgroups_train, 1, 100)[1].shape)

In [ ]:
learning_rate =0.001
training_epochs = 50
batch_size = 150
display_step = 1

n_hidden_1 = 100
n_hidden_2 = 100
n_input = total_words
n_classes = 3

In [ ]:
input_tensor = tf.placeholder(tf.float32, [None, n_input], name='input')
output_tensor = tf.placeholder(tf.float32, [None, n_classes], name='output')

In [ ]:
def multilayer_perceptron(input_tensor, weights, biases):
    layer_1_multiplication = tf.matmul(input_tensor, weights['h1'])
    layer_1_addition = tf.add(layer_1_multiplication, biases['b1'])
    layer_1 = tf.nn.relu(layer_1_addition)
    
    layer_2_multiplication = tf.matmul(layer_1, weights['h2'])
    layer_2_addition = tf.add(layer_2_multiplication, biases['b2'])
    layer_2 = tf.nn.relu(layer_2_addition)
    
    out_layer_multiplication = tf.matmul(layer_2, weights['out'])
    out_layer_addition = out_layer_multiplication + biases['out']
    
    return out_layer_addition

In [ ]:
weights = {
    'h1': tf.Variable(tf.random.normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random.normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random.normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random.normal([n_hidden_1])),
    'b2': tf.Variable(tf.random.normal([n_hidden_2])),
    'out': tf.Variable(tf.random.normal([n_classes]))
}

prediction = multilayer_perceptron(input_tensor, weights, biases)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = prediction, labels = output_tensor))

optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

init = tf.global_variables_initializer()

saver = tf.train.Saver()



In [ ]:
training_epochs = 50
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(newsgroups_train.data)/batch_size)
        
        for i in range(total_batch):
            batch_x, batch_y = get_batch(newsgroups_train, i, batch_size)
            c, _ = sess.run([loss, optimizer], feed_dict={input_tensor: batch_x, output_tensor:batch_y})
            avg_cost += c / total_batch
            if avg_cost < 0.5:
                epoch = training_epochs
                break
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "loss=", "{:0.9f}".format(avg_cost))
    
    print("Optimization Finished!")
    
    correct_prediction = tf.equal(tf.argmax(prediction, 1), tf.argmax(output_tensor, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    total_test_data = len(newsgroups_test.target)
    batch_x_test, batch_y_test = get_batch(newsgroups_test, 0, total_test_data)
    print("Accuracy:", accuracy.eval({input_tensor: batch_x_test, output_tensor: batch_y_test}))
    
    save_path = saver.save(sess, "/tmp/model.ckpt")
    print("Model saved in path: %s" % save_path)

In [ ]:
text_for_prediction = newsgroups_test.data[5]
print('text:', text_for_prediction)

In [ ]:
print("text correct category:", newsgroups_test.target[5])

In [ ]:
vector_txt = text_to_vector(text)
input_array = np.array([vector_txt])

In [ ]:
input_array.shape

In [ ]:
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print('Model restored.')
    
    classification = sess.run(tf.argmax(prediction, 1), feed_dict={input_tensor: input_array})
    print('Predicted category:', classification)

In [ ]:
x_10_texts,y_10_correct_labels = get_batch(newsgroups_test,0,10)

saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess, "/tmp/model.ckpt")
    print("Model restored.")
    
    classification = sess.run(tf.argmax(prediction, 1), feed_dict={input_tensor: x_10_texts})
    print("Predicted categories:", classification)

In [ ]:
print(np.argmax(y_10_correct_labels, 1))